### Country Automation

To do the country automation, a priority lottery will be utilized

#### Requirements
1. Pandas - python library
2. CSV of schools, delegate numbers, entries, countries, country positions, characters

In [2]:
# Import necessary libraries, get setup
import pandas as pd
import numpy as np
from random import randint
from random import shuffle

data = pd.read_csv('./data.csv')
data[0:10]

,Schools,Delegates,Entries,Countries,Positions,Characters
0,School 1,34.0,3.0,Country 1,3,Character 1
1,School 2,11.0,3.0,Country 2,11,character 2
2,School 3,14.0,3.0,Country 3,11,Character 3
3,School 4,19.0,3.0,Country 4,12,Character 4
4,School 5,23.0,2.0,Country 5,4,Character 5
5,School 6,27.0,5.0,Country 6,2,Character 6
6,School 7,1.0,2.0,Country 7,4,Character 7
7,School 8,31.0,1.0,Country 8,7,Character 8
8,School 9,13.0,4.0,Country 9,10,Character 9
9,School 10,27.0,1.0,Country 10,12,Character 10


### Setup school data: grab schools, delegates, entries and drop the NaN

In [3]:
schools_arr = data[['Schools','Delegates','Entries']].dropna()
schools = []
for i in range(schools_arr.shape[0]):
    schools.append(
        {'school': str(schools_arr.at[i,'Schools']), 
         'delegates': int(schools_arr.at[i,'Delegates']),
         'entries': int(schools_arr.at[i,'Entries']),
         'remaining': int(schools_arr.at[i,'Delegates']),
         'countries': [], 'characters': []
        }
    )
schools[0:3]

[{'characters': [],
  'countries': [],
  'delegates': 34,
  'entries': 3,
  'remaining': 34,
  'school': 'School 1'},
 {'characters': [],
  'countries': [],
  'delegates': 11,
  'entries': 3,
  'remaining': 11,
  'school': 'School 2'},
 {'characters': [],
  'countries': [],
  'delegates': 14,
  'entries': 3,
  'remaining': 14,
  'school': 'School 3'}]

### Setup country data: country name and position

In [4]:
country_arr = data[['Countries','Positions']].dropna()
countries = []
for i in range(country_arr.shape[0]):
    countries.append({'country': str(country_arr.at[i,'Countries']),'positions': int(country_arr.at[i,'Positions'])})
countries[0:3]

[{'country': 'Country 1', 'positions': 3},
 {'country': 'Country 2', 'positions': 11},
 {'country': 'Country 3', 'positions': 11}]

### Setup character name: character name

In [5]:
char_arr = data[['Characters']].dropna()
characters = []
for i in range(char_arr.shape[0]):
    characters.append({'character': str(char_arr.at[i,'Characters'])})
characters[0:3]

[{'character': 'Character 1'},
 {'character': 'character 2'},
 {'character': 'Character 3'}]

### Setup the priority 'hat'. Put several of each school name in the hat (determined by entry number)

In [6]:
# Create the priority lottery array
school_priority_list = []
for entry in schools:
    school_priority_list += [entry['school']] * entry['entries']
shuffle(school_priority_list)
school_priority_list[0:3]

['School 17', 'School 21', 'School 18']

### Now, we have the data prepared; define some functions to check if we are done; first constants

In [7]:
percentageCountry = 0.80
schoolsOnNextStage = []
assignAttemps = 25

### doneCountries() will determine if we are finished the country stage
### assignCountry() will assign a country to a school (the algorithm)
### findIndexOfSchool() will use schoolName to find school object in the schools array
### removeFromPriority() will remove all the entries of a school, once its has enough country assignments

In [8]:
def doneCountries(currIteration,maxIterations):
    if currIteration >= maxIterations:
        return True
    if len(schools) == 0:
        return True
    if len(school_priority_list) == 0:
        return True
    if len(countries) == 0:
        return True
    return False

def assignCountry():
    country = countries[0]
    for i in range(assignAttemps):
        randInt = randint(0, len(school_priority_list) - 1) 
        schoolName = school_priority_list[randInt]
        ind = findIndexOfSchool(schoolName)
        if schools[ind]['remaining'] < country['positions']:
            continue
        else:
            schools[ind]['countries'].append(countries.pop(0))
            schools[ind]['remaining'] = schools[ind]['remaining'] - country['positions']
            if schools[ind]['remaining'] <= int((1.00-percentageCountry)*schools[ind]['delegates']):
                schoolsOnNextStage.append(schools.pop(ind))
                removeFromPriority(schoolName)
            return
            
    # if the function gets here, it wasn't able to assign the country: move it to the end
    countries.append(country)
    countries.pop(0)
        
def findIndexOfSchool(schoolName):
    for i in range(len(schools)):
        if schools[i]['school'] == schoolName:
            return i
    raise Exception("Can't find " + schoolName + " in the list")
    
def removeFromPriority(schoolName):
    i = 0
    while i < len(school_priority_list):
        if school_priority_list[i] == schoolName:
            school_priority_list.pop(i)
        else:
            i = i + 1

### algorithmCountries() loop to assign countries; if there are remaining schools, add them to the schoolsOnNextStage array at the end

In [9]:
def algorithmCountries():
    maxIterations = 10000
    currIteration = 0
    while not doneCountries(currIteration,maxIterations):
        assignCountry()
        currIteration = currIteration + 1
    for school in schools:
        schoolsOnNextStage.append(school)

In [10]:
algorithmCountries()
schoolsOnNextStage[0:3]

[{'characters': [],
  'countries': [{'country': 'Country 6', 'positions': 2},
   {'country': 'Country 8', 'positions': 7}],
  'delegates': 11,
  'entries': 3,
  'remaining': 2,
  'school': 'School 2'},
 {'characters': [],
  'countries': [{'country': 'Country 17', 'positions': 6}],
  'delegates': 6,
  'entries': 5,
  'remaining': 0,
  'school': 'School 22'},
 {'characters': [],
  'countries': [{'country': 'Country 14', 'positions': 7},
   {'country': 'Country 28', 'positions': 10}],
  'delegates': 18,
  'entries': 4,
  'remaining': 1,
  'school': 'School 13'}]

In [11]:
countries[0:3]

[{'country': 'Country 86', 'positions': 9},
 {'country': 'Country 89', 'positions': 9},
 {'country': 'Country 90', 'positions': 5}]

In [12]:
finishedSchools = []

def doneCharacters(currIteration,maxIterations):
    if currIteration >= maxIterations:
        return True
    if len(schoolsOnNextStage) == 0:
        return True
    if len(characters) == 0:
        return True
    return False

def assignCharacters():
    if len(schoolsOnNextStage) == 0:
        return
    
    randInt = randint(0, len(schoolsOnNextStage) - 1)
    if schoolsOnNextStage[randInt]['remaining'] == 0:
        return
    else:
        schoolsOnNextStage[randInt]['characters'].append(characters.pop(0))
        schoolsOnNextStage[randInt]['remaining'] = schoolsOnNextStage[randInt]['remaining'] - 1

def removeZeroRemaining():
    i = 0
    while i < len(schoolsOnNextStage):
        if schoolsOnNextStage[i]['remaining'] == 0:
            finishedSchools.append(schoolsOnNextStage.pop(i))
        else:
            i = i + 1
        
def algorithmCharacters():
    maxIterations = 10000
    currIteration = 0
    while not doneCharacters(currIteration,maxIterations):
        removeZeroRemaining()
        assignCharacters()
        currIteration = currIteration + 1
    for school in schoolsOnNextStage:
        finishedSchools.append(school)
        
algorithmCharacters()
finishedSchools[0:3]

[{'characters': [],
  'countries': [{'country': 'Country 17', 'positions': 6}],
  'delegates': 6,
  'entries': 5,
  'remaining': 0,
  'school': 'School 22'},
 {'characters': [],
  'countries': [{'country': 'Country 3', 'positions': 11},
   {'country': 'Country 38', 'positions': 5}],
  'delegates': 16,
  'entries': 3,
  'remaining': 0,
  'school': 'School 21'},
 {'characters': [],
  'countries': [{'country': 'Country 2', 'positions': 11},
   {'country': 'Country 39', 'positions': 3}],
  'delegates': 14,
  'entries': 3,
  'remaining': 0,
  'school': 'School 3'}]

In [13]:
characters[0:3]

[{'character': 'Character 43'},
 {'character': 'Character 44'},
 {'character': 'Character 45'}]

In [14]:
with open('output.csv','w') as fp:
    fp.write("School,Delegates,Remaining\n")
    for school in finishedSchools:
        line = ""
        line += school['school'] + ','
        line += str(school['delegates']) + ','
        line += str(school['remaining']) + ','
        for country in school['countries']:
            line += country['country'] + " (" + str(country['positions']) + "),"
        for character in school['characters']:
            line += character['character'] + ","
        if line.endswith(','):
            line = line[:-1] + "\n"
        else:
            line = line + "\n"
        fp.write(line)